<a href="https://colab.research.google.com/github/strongbyson/Quant-Projects/blob/main/Final_vers_Gap_up%20%26%20%25b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install technical-analysis
!pip install ta
!pip install backtrader

import pandas as pd
import numpy as np
import ta as ta
import backtrader as bt

In [124]:
# Load the CSV file
nb = pd.read_csv('niftybees.csv', index_col = 'Date', parse_dates=True)
#print(nb.head(10))

# Initialize trade parameters
initial_capital = 1000000
available_capital = initial_capital * 0.75
trade_counter = 0

# Calculate Bollinger bands & %b
from technical_analysis import overlays
nb["bband_lower"], nb["bband_upper"] = overlays.bbands(nb.Close, period=20)
nb["pct_b"] = (nb.Close - nb["bband_lower"]) / (nb["bband_upper"] - nb["bband_lower"])
#print(nb.head(21))

# Initialize columns for trade details
nb['Trade Counter'] = np.nan
nb['Buying Price'] = np.nan
nb['Units Bought'] = np.nan
nb['Buying Amount'] = np.nan
nb['Available Capital'] = np.nan
nb['Selling Price'] = np.nan
nb['Selling Amount'] = np.nan
nb['Profit'] = np.nan

# Iterate through the dataframe to execute the trading strategy
for i in range(1, len(nb)):
  current_close = nb['Close'].iloc[i]
  current_open = nb['Open'].iloc[i]
  prev_close = nb['Close'].iloc[i-1]
  current_pct_b = nb['pct_b'].iloc[i]
  prev_pct_b = nb['pct_b'].iloc[i-1]

  # Buying condition
  if trade_counter == 0 and current_pct_b > 0.4 and current_pct_b > prev_pct_b:
    buying_price = current_close
    units_bought = round((available_capital / current_close),0) - 1
    buying_amount = units_bought * current_close
    available_capital -= buying_amount
    trade_counter = 1

    nb.loc[nb.index[i], 'Trade Counter'] = trade_counter
    nb.loc[nb.index[i], 'Available Capital'] = available_capital
    nb.loc[nb.index[i], 'Units Bought'] = units_bought
    nb.loc[nb.index[i], 'Buying Price'] = buying_price
    nb.loc[nb.index[i], 'Buying Amount'] = buying_amount

  # Selling condition
  elif trade_counter == 1:
    selling_price = current_open
    selling_amount = units_bought * selling_price
    profit = selling_amount - buying_amount
    available_capital += selling_amount
    units_bought -= units_bought
    trade_counter = 0
    units_bought = 0

    nb.loc[nb.index[i], 'Units Bought'] = units_bought
    nb.loc[nb.index[i], 'Selling Price'] = selling_price
    nb.loc[nb.index[i], 'Selling Amount'] = selling_amount
    nb.loc[nb.index[i], 'Profit'] = profit

  else:
    pass

print(nb.head(40))
nb.to_csv('niftybees_export.csv', index=False)


             Open    High     Low   Close  bband_lower  bband_upper     pct_b  \
Date                                                                            
3/25/2019  118.20  119.50  118.20  118.81          NaN          NaN       NaN   
3/26/2019  124.00  124.00  119.00  120.39          NaN          NaN       NaN   
3/27/2019  120.00  121.00  119.92  120.10          NaN          NaN       NaN   
3/28/2019  120.10  121.50  120.10  120.98          NaN          NaN       NaN   
3/29/2019  125.50  125.50  121.07  122.06          NaN          NaN       NaN   
04/01/19   122.40  123.26  121.80  122.08          NaN          NaN       NaN   
04/02/19   120.30  122.70  120.30  122.53          NaN          NaN       NaN   
04/03/19   122.60  123.30  122.00  122.15          NaN          NaN       NaN   
04/04/19   122.30  122.33  121.51  121.81          NaN          NaN       NaN   
04/05/19   123.50  126.50  121.50  122.50          NaN          NaN       NaN   
04/08/19   122.50  122.79  1